<a href="https://colab.research.google.com/github/josephedward/Libridex_Recommendation_Engine/blob/master/Test_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install --upgrade pip
!pip install flask
!pip install pandas
!pip install bs4
!pip install splinter
!pip install nltk
!pip install sklearn
!pip install gensim
!pip install matplotlib



Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.1.1)


In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import time 
from splinter import Browser
from textblob import TextBlob
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline
import warnings;
import requests
from google.colab import files
# import untangle
# warnings.filterwarnings('ignore')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
                   
            # Function for removing NonAscii characters
def _removeNonAscii(s):
              return "".join(i for i in s if  int(i)<128)# Function for converting into lower case
def make_lower_case(text):
              return str(text).lower()# Function for removing stop words
def remove_stop_words(text):
              text = text.split()
              stops = set(stopwords.words("english"))
              text = [w for w in text if not w in stops]
              text = " ".join(text)
              return text# Function for removing punctuation
def remove_punctuation(text):
              tokenizer = RegexpTokenizer(r'\w+')
              text = tokenizer.tokenize(text)
              text = " ".join(text)
              return text
#Function for removing the html tags
def remove_html(text):
              html_pattern = re.compile('<.*?>')
              return html_pattern.sub(r'', text)# Applying all the functions in description and storing as a cleaned_desc

print("cleaning book list")
book_df = pd.read_csv('https://raw.githubusercontent.com/josephedward/Libridex_Recommendation_Engine/master/notebooks/lbvx_book_list_072420')
book_df['cleaned_desc'] = book_df['description'].apply(func = make_lower_case)
# testdf['cleaned_desc'] = df.cleaned_desc.apply(func = make_lower_case)
book_df['cleaned_desc'] = book_df.cleaned_desc.apply(func = remove_stop_words)
book_df['cleaned_desc'] = book_df.cleaned_desc.apply(func=remove_punctuation)
book_df['cleaned_desc'] = book_df.cleaned_desc.apply(func=remove_html)




rec_whole_objs=[]

            
            
            
for x in range(16000):
    # x=47
    print("Searching Librivox ID: ",x)
    librivox_id_search = f'https://librivox.org/api/feed/audiobooks/?id={x}'            
# Use requests to retrieve data from a given URL
# Parse the whole HTML page using BeautifulSoup
    # librivox_id_search = 'https://librivox.org/api/feed/audiobooks/?id=52'
    try:
      lbvx_response = requests.get(librivox_id_search)
      soup = BeautifulSoup(lbvx_response.text, 'html.parser')
      lib_id=soup.find("id").get_text()
      title=soup.find("title").get_text()
      description=soup.find("description").get_text()
      language=soup.find("language").get_text()
      copyright_year=soup.find("copyright_year").get_text()
      lib_book_url=soup.find('url_librivox').get_text()
      author_fname= soup.find('author').find('first_name').get_text()
      author_lname= soup.find('author').find('last_name').get_text()
      author= author_fname+" "+author_lname
      book_page_resp = requests.get(lib_book_url)
      soup = BeautifulSoup(book_page_resp.text, 'html.parser')
      genre=soup.find_all('p', class_='book-page-genre')[0].get_text()
      genre_arr=genre.split(":")
      genre=genre_arr[1]
      img_url=soup.find('div', class_="book-page-book-cover").find('img').get('src')
      # book_objs.append
      book_scrape_df = pd.DataFrame([{"lib_id":lib_id,
                  "title":title,
                  "author":author,
                  "genre":genre,
                  "description":description,
                  "language":language,
                  "copyright_year":copyright_year,
                  "lib_book_url":lib_book_url                  
                      }])
      print(book_scrape_df)
      runFlag = True
    except:
      print("couldn'y locate : ",x )
      runFlag = False
    # Matching the language with the dataset and reset the index
    print("running analyis on: ", title)
    if runFlag==True:
      try:
        data = book_df.loc[book_df['language'] == "English"]  
        # data = 
        data.reset_index(level = 0, inplace = True) 
        print('Convert the index into series')
        indices = pd.Series(data.index, index = data['title'])
        # print(indices)
        print("Converting the book title into vectors and used bigram")
        tf = TfidfVectorizer(analyzer='word', ngram_range=(2, 2), min_df = 1, stop_words='english')
        # print(tf)
        tfidf_matrix = tf.fit_transform(data['cleaned_desc'])
        # print(tfidf_matrix)
        print("Calculating the similarity measures based on Cosine Similarity")
        sg = cosine_similarity(tfidf_matrix, tfidf_matrix)
        # print(sg)    
        print("Get the index corresponding to original_title")       
        idx = indices[title]
        # print(idx)
        print('Get the pairwise similarity scores') 
        sig = list(enumerate(sg[idx]))
        # print(sig)
        print("Sort the books")
        sig = sorted(sig, key=lambda x: x[1].any(), reverse=True)
        print('Scores of the 5 most similar books') 
        sig = sig[1:6]
        # print(sig)
        print("Book indicies")
        bk_indices = [i[0] for i in sig]
        print("Top 5 book recommendations")
        # rec = data['lib_url'].iloc[bk_indices]   
        rec=data.iloc[bk_indices]
        print("REC.COLUMNS : ")
        print(rec.columns) 
        print("Title: ", book_scrape_df)
    
        rec_arr = rec
        print("Rec Arr: ", rec_arr)

        book_scrape_df['Rec_Info_Arr']= rec_arr.to_json(orient='records')        
        # book_scrape_df.to_csv("./denormalized_scrape_match_v1.csv")
        rec_whole_objs.append(book_scrape_df.to_json(orient='records'))        
        rec_whole_objs_df = pd.DataFrame(rec_whole_objs)
        rec_whole_objs_df.to_csv("./denormalized_scrape_match_v2.csv")
      except:
        print("couldn't analyze: ", title)
        # break



files.download('./denormalized_scrape_match_v1.csv')
# static_recs_path="../resources/combined_recommendations_v1.csv"
# recs_df=pd.read_csv(static_recs_path)

# rec_objs_list=[]

# for x in recs_df['book_title']:
#     rec_arr = recs_df.loc[recs_df['book_title'] == x]['book_recommendation_urls'].values
#     title = x


    
#     pattern=r'(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))';
#     print("matching URLS...")
#     match = re.findall(pattern, str(rec_arr))
#     # match[1][0]
#     rec_urls=[]
#     for x in match:
#         rec_urls.append(x[0])

#     # Scrape
    # for x in rec_arr.itertuples():
    #         print("scraping :", x.title)
    #         x_req = requests.get(x.lib_url)
    #         search_page = x_req.text
    #         soup = BeautifulSoup(search_page, 'html.parser')
    #         book_page= soup.find('div', class_="page book-page")
    #         # title=book_page.find('h1').get_text()
    #         # author=book_page.find('p', class_="book-page-author").get_text()
    #         img_url=book_page.find('div', class_="book-page-book-cover").find('img').get('src')
    #         rec_whole_objs.append({
    #             'lib_id':x.lib_id,
    #             'title':x.title,
    #             'author':x.author,
    #             'lib_url':img_url,
    #             'page_url':x.lib_url,
    #             'genre':x.genre,
    #             'description':x.description,
    #             'language':x.language,
    #             'copyright_year'x.copyright_year, 
    #             'cleaned_desc':x.cleaned_desc
    #         })
    # print("done scrapin recs fer : ", title)
    # print(rec_objs)
    # break
#     rec_objs_list.append({
#                 "title":title,
#                 "rec_objects":rec_objs
#             })
#     print("appended")
#     rec_objs_df=pd.DataFrame(rec_objs_list)
#     rec_objs_df.to_csv('rec_obj_list_v1.csv')
#     print("written to csv")


# print("finished")
# print(rec_objs_list)


# from google.colab import files
# df.to_csv('filename.csv') 
# files.download('filename.csv')

# # rec = data[['title']].iloc[movie_indices]
# # rec_whole_df = book_df[book_df['title'].isin(rec['title'])]
# # print(rec_whole_df)

Streaming output truncated to the last 5000 lines.
Book indicies
Top 5 book recommendations
REC.COLUMNS : 
Index(['index', 'Unnamed: 0', 'lib_id', 'title', 'author', 'genre',
       'description', 'language', 'copyright_year', 'lib_url', 'cleaned_desc'],
      dtype='object')
Title:    lib_id  ...                                   lib_book_url
0    453  ...  https://librivox.org/us-historical-documents/

[1 rows x 8 columns]
Rec Arr:      index  ...                                       cleaned_desc
15     17  ...  p among banned books united states uncle tom s...
33     38  ...  p if like jane austen probably like book p p m...
57     68  ...  weekly poem flag day 2006 tells largely apocry...
72     86  ...  p em declaration independence em document thir...
93    114  ...  p the amendments constitution usa continuously...

[5 rows x 11 columns]
Searching Librivox ID:  454
  lib_id  ...                                       lib_book_url
0    454  ...  https://librivox.org/ueber-die-wei

In [10]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
